# Generative Agents Example

This code show the concept of Generative Agents with LLMs, described by [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442). 

Links:
* [LangChain implementation](https://github.com/langchain-ai/langchain/blob/master/cookbook/generative_agents_interactive_simulacra_of_human_behavior.ipynb)
* [GenerativeAgent implementation](https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/generative_agents/generative_agent.py)
* [sqlite-vss](https://github.com/asg017/sqlite-vss)


![](../assets/generative-agents.jpeg)


In [2]:
import os
import math
import faiss
from datetime import datetime, timedelta
from typing import List
from getpass import getpass
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import SQLiteVSS
from termcolor import colored

from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

In [3]:
os.environ["AZURE_OPENAI_API_KEY"] = getpass("Enter your Azure OpenAI API Key: ")
os.environ["AZURE_OPENAI_ENDPOINT"] = getpass("Enter your Azure OpenAI API Endpoint: ")

In [4]:
# The name you want to use when interviewing the agent.

USER_NAME = "Person A"  
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4",
    max_tokens=1500
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-05-15",
)

In [5]:
set_llm_cache(SQLiteCache(database_path=".generative-agents.db"))

In [8]:
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = SQLiteVSS.create_connection(db_file="generative-agent-retriever.db")
    
    vectorstore = SQLiteVSS(
        table="generative_agent", 
        embedding=embeddings.embed_query, 
        connection=index,       
    )

    #index = faiss.IndexFlatL2(embedding_size)
    #vectorstore = FAISS(
    #    embeddings.embed_query,
    #    index,
    #    InMemoryDocstore({}),
    #    {},
    #    relevance_score_fn=relevance_score_fn,
    #)
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [ ]:
tommies_memory = GenerativeAgentMemory(
    llm=llm,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=llm,
    memory=tommies_memory,
)

In [11]:
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
The text does not provide any statements about Tommie's core characteristics.


In [12]:
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [13]:
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is observant, nostalgic, and has physical needs such as hunger and rest. He also appears to be on a long journey, as indicated by his tiredness from driving.


In [14]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [15]:
interview_agent(tommie, "What do you like to do?")

'Tommie said "I really like design. I enjoy looking at different structures and imagining how they were put together. It\'s a great way to stimulate my creativity. I also love reminiscing about my past, like my childhood pet, Bruno. He was such a good dog. How about you? What do you enjoy doing?"'

In [16]:
interview_agent(tommie, "What are you looking forward to doing today?")

'Tommie said "Well, since I\'m currently looking for a job, I\'m hoping to browse some job listings and send out a few applications today. I also plan to take some time to rest and maybe grab a bite to eat since I\'m feeling quite hungry. And with the new neighbors having a cat, I might go over and say hello, it might remind me of my pet Bruno. What about you? Any plans for today?"'

In [18]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "I guess my biggest worry today is finding a job. I\'m anxious about the whole process, from searching for the right listings to sending applications and waiting for responses. Plus, I\'m not in a great physical state as I\'m quite tired and hungry. But I\'m trying to stay positive and hopeful. What about you? Is there something you\'re particularly worried about today?"'

In [17]:
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [22]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie gets up and closes the window to block out the construction noise.
Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie brews a pot of coffee and pours himself a cup, enjoying the aroma.
Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some. Tommie finds some coffee filters in one of his moving boxes and breathes a sigh of relief.
Tommie finally finds the filters and makes himself a cup of coffee. Tommie sips his coffee, the familiar taste bringing a small comfort amidst the chaos of the move and job hunting.
The coffee tastes bitter, and Tommie regrets not buying a better brand. Tommie makes a mental note to buy a better brand of coffee next time.
Tommie checks his email and sees that he has no job offers yet. Tommie feels a pang of disappointment but reminds himself to stay patient and persistent in his job search.
To

KeyboardInterrupt: 

In [23]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a roller coaster today. My job search is ongoing which can be quite exhausting and frustrating at times. I did manage to drop off my resume at a few places, and even applied for a job online. I also took a break to go for a walk in the park which was really calming. I played some frisbee with some people, got hit in the face, but it was a fun distraction. Oh, and I met a friendly dog which reminded me of Bruno, my childhood pet. How about your day?"'